In [1]:
#Install required libraries
!pip install keras==2.8
!pip install h5py
!pip install -U keras-tuner

  Using cached keras_tuner-1.1.2-py3-none-any.whl (133 kB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)


In [2]:
#import for hdf5 table reading
import h5py
import numpy as np
import pandas as pd
import sklearn
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import os
import time
import math
LOG_DIR = f"{int(time.time())}"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
#import tensorflow
import tensorflow as tf
tf.__version__

'2.8.0'

In [4]:
 #import keras and Sequential
from tensorflow import keras 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
keras.__version__

'2.8.0'

In [5]:
 #Next 3 cells of code populate the backgroundVar and higgsVar table with desired data for background pattern
background = pd.HDFStore('background.hdf5', mode = 'r')
higgs = pd.HDFStore('higgs.hdf5', mode = 'r')

In [6]:
#Notes, could not find mJ1 and there were double dRB1Ph listed in README
backgroundVar = background['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]
higgsVar = higgs['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]

In [7]:
backgroundVar['isBackground'] = 1
higgsVar['isBackground'] = 0

In [8]:
backgroundValid, backgroundTrain = backgroundVar[:1000000], backgroundVar[1000000:] 
higgsValid, higgsTrain = higgsVar[:1000000], higgsVar[1000000:]
backgroundTrain = backgroundTrain[:1000000]
higgsTrain = higgsTrain[:1000000]


In [9]:
 keras.backend.clear_session()
np.random.seed(50)
tf.random.set_seed(50)

In [10]:
train_labels = pd.concat([backgroundTrain['isBackground'],higgsTrain['isBackground']])
valid_labels = pd.concat([backgroundValid['isBackground'],higgsValid['isBackground']])
#train_labels

In [11]:
train_inputs = pd.concat([backgroundTrain[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']],higgsTrain[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]])
valid_inputs = pd.concat([backgroundValid[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']],higgsValid[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]])
train_inputs = train_inputs.reset_index(drop=True)
valid_inputs = valid_inputs.reset_index(drop=True)
# demonstrate data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# fit scaler on data
scaler.fit(train_inputs)
scaler.fit(valid_inputs)
# apply transform
train = scaler.transform(train_inputs)
valid = scaler.transform(valid_inputs)

In [12]:
train_labels = train_labels.reset_index(drop=True)
valid_labels = valid_labels.reset_index(drop=True)
initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

In [16]:
def step_decay(epoch):
   initial_lrate = 0.05
   drop = 0.8
   epochs_drop = 2.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
class LearningRate(tf.keras.callbacks.Callback):
  
  def on_train_begin(self,logs={}):
    self.lr_epoch=[]

  def on_epoch_end(self, batch, logs={}):
    self.lr_epoch.append(step_decay(len(self.lr_epoch)+1))

lr_history=LearningRate()

In [18]:
model = keras.models.Sequential([(Dense(12,activation = 'relu', kernel_initializer=initializer)),
                                 tf.keras.layers.BatchNormalization(),
                                 (Dense(9,activation = 'relu', kernel_initializer=initializer)),
                                 tf.keras.layers.BatchNormalization(),
                                 (Dense(6, activation='sigmoid', kernel_initializer=initializer)),
                                 tf.keras.layers.BatchNormalization(),
                                 (Dense(3, activation='relu', kernel_initializer=initializer)),
                                 tf.keras.layers.BatchNormalization(),
                                 (Dense(1, activation='sigmoid', kernel_initializer=initializer, name = "predictions"))
                                ])

model.compile(optimizer=keras.optimizers.Nadam(), loss='binary_crossentropy', metrics=['accuracy'])

history=model.fit(train, train_labels, epochs=10, callbacks=[lr_history,lrate], validation_data=(valid, valid_labels),shuffle=True, verbose=1)
accuracy = model.evaluate(valid, valid_labels)

Epoch 1/10
45374/45374 [==============================] - 720s 16ms/step - loss: 0.5404 - accuracy: 0.7390 - val_loss: 0.9571 - val_accuracy: 0.5000 - lr: 0.0500
Epoch 2/10
45374/45374 [==============================] - 670s 15ms/step - loss: 0.5381 - accuracy: 0.7399 - val_loss: 0.6031 - val_accuracy: 0.6760 - lr: 0.0400
Epoch 3/10
45374/45374 [==============================] - 688s 15ms/step - loss: 0.5372 - accuracy: 0.7408 - val_loss: 0.6076 - val_accuracy: 0.6744 - lr: 0.0400
Epoch 4/10
45374/45374 [==============================] - 698s 15ms/step - loss: 0.5381 - accuracy: 0.7401 - val_loss: 0.8663 - val_accuracy: 0.5000 - lr: 0.0320
Epoch 5/10
45374/45374 [==============================] - 700s 15ms/step - loss: 0.5360 - accuracy: 0.7414 - val_loss: 0.6217 - val_accuracy: 0.6708 - lr: 0.0320
Epoch 6/10
45374/45374 [==============================] - 704s 16ms/step - loss: 0.5346 - accuracy: 0.7421 - val_loss: 0.7932 - val_accuracy: 0.5715 - lr: 0.0256
Epoch 7/10
45374/45374 [====

In [ ]:
#tuner searches for the best version of the model
#def build_model(hp):
#    model1 = keras.models.Sequential()
    
#    model1.add(Dense(12,activation = "tanh"))
#    model1.add(Dropout(0.05))
    
    #layers
#    for i in range(hp.Int('n_layers', 1, 6)):
#        model1.add(Dense(hp.Int(f'conv_{i}_units', min_value = 1, max_value = 50, step = 1)))
#        model1.add(Activation('tanh'))
#        model1.add(Dropout(0.05))
    
    #final layer
#    model1.add(Dense(1, activation="sigmoid", name = "predictions"))
    
    #compile
#    model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy"])
#    return model1

#tuner = RandomSearch(
#    build_model,
#    objective = "accuracy",
#    max_trials = 100,
#    executions_per_trial = 1,
#    directory = LOG_DIR)

#tuner.search(x=train,
#             y=train_labels,
#             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
#             epochs=10,
#             batch_size=50,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
#            )